In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [4]:
df=pd.read_csv('/content/hindi_tr.csv')
df

,tweet_id,text,task1,task2,ID
0,1.127755e+18,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HOF,HATE,hasoc_2020_hi_2250
1,1.123579e+18,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT,NONE,hasoc_2020_hi_381
2,1.127750e+18,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HOF,HATE,hasoc_2020_hi_1510
3,1.127661e+18,Yey nina khothani labafazi benu phambili Finis...,HOF,PRFN,hasoc_2020_hi_1010
4,1.123487e+18,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HOF,HATE,hasoc_2020_hi_331
...,...,...,...,...,...
2958,1.127651e+18,@Varun_dvn जिसके नाम के साथ इंडिअनस लगा हो उसक...,NOT,NONE,hasoc_2020_hi_2781
2959,1.127752e+18,मुझे सबसे ज्यादा राहुल और सोनियाके बारे बुरा ल...,NOT,NONE,hasoc_2020_hi_2283
2960,1.123722e+18,RT @Gujju_Er: एक आतंकी जिसे कांग्रेस ने\n'शांत...,HOF,HATE,hasoc_2020_hi_2022
2961,1.127753e+18,RT @dayanand151: @jafahar1 @BBCHindi @Hariramk...,HOF,OFFN,hasoc_2020_hi_882


In [5]:
X=df[['text','task2']]
X.head()

,text,task2
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NONE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,PRFN
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HATE


In [6]:
df1=pd.read_csv('/content/hindi_test_1509.csv')
df1

,tweet_id,text,task1,task2,ID
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,HOF,OFFN,hasoc_2020_hi_1938
1,1127492853171535872,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NOT,NONE,hasoc_2020_hi_1201
2,1123638375896903681,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NOT,NONE,hasoc_2020_hi_1628
3,1127496460310683648,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NOT,NONE,hasoc_2020_hi_3257
4,1127667277535399937,@manakgupta बादल....😂😂😂,NOT,NONE,hasoc_2020_hi_4133
...,...,...,...,...,...
658,1123814066907242503,RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...,NOT,NONE,hasoc_2020_hi_466
659,1127616035689979904,RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...,HOF,NONE,hasoc_2020_hi_1758
660,1127502823044702208,@CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...,HOF,OFFN,hasoc_2020_hi_3521
661,1127780829914992640,RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...,NOT,NONE,hasoc_2020_hi_1293


In [7]:
Y=df1[['text','task2']]
Y.head(20)

,text,task2
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,OFFN
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NONE
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NONE
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NONE
4,@manakgupta बादल....😂😂😂,NONE
5,@manakgupta देखो भक्त एंकरों की पोस्ट,NONE
6,RT @nehapatel342634: जब भी मेरे भाई के दोस्त क...,PRFN
7,"RT @iVinayShukla_: ➡️IBM 1951 में भारत आयी थी,...",NONE
8,अब केवल दो चरण बचे हैं अपने दोनों चरणों को मतद...,NONE
9,@DrKumarVishwas @AmritanjaliAbh1 @ImranKhanPTI...,NONE


In [8]:
cols=['task2']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
X.head()

,text,task2
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,0
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",1
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,0
3,Yey nina khothani labafazi benu phambili Finis...,2
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,0


In [10]:
Y.head()

,text,task2
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,0
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,1
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",1
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,1
4,@manakgupta बादल....😂😂😂,1


In [11]:
print(X['task2'].value_counts())
print(Y['task2'].value_counts())



1    2116
3     465
0     234
2     148
Name: task2, dtype: int64
1    493
0     87
3     56
2     27
Name: task2, dtype: int64


In [12]:
X_train=X['text']
X_train

0       1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...
1       RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...
2       RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...
3       Yey nina khothani labafazi benu phambili Finis...
4       RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...
                              ...                        
2958    @Varun_dvn जिसके नाम के साथ इंडिअनस लगा हो उसक...
2959    मुझे सबसे ज्यादा राहुल और सोनियाके बारे बुरा ल...
2960    RT @Gujju_Er: एक आतंकी जिसे कांग्रेस ने\n'शांत...
2961    RT @dayanand151: @jafahar1 @BBCHindi @Hariramk...
2962    RT @manakgupta: “हमें 5 साल हो गए, हमने संविधा...
Name: text, Length: 2963, dtype: object

In [13]:
y_train=X['task2']
y_train

0       0
1       1
2       0
3       2
4       0
       ..
2958    1
2959    1
2960    0
2961    3
2962    1
Name: task2, Length: 2963, dtype: int64

In [14]:
X_test=Y['text']
X_test

0      @Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...
1      RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...
2      RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...
3      @BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...
4                                @manakgupta बादल....😂😂😂
                             ...                        
658    RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...
659    RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...
660    @CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...
661    RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...
662    @athavale_abhi @kavita_tewari गांधी परिवार और ...
Name: text, Length: 663, dtype: object

In [15]:
y_test=Y['task2']
y_test

0      0
1      1
2      1
3      1
4      1
      ..
658    1
659    1
660    0
661    1
662    3
Name: task2, Length: 663, dtype: int64

In [16]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2963,), (663,), (2963,), (663,))

In [17]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [18]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [1, आदमीं, को, मारने, पर, गोडसे, आतंकी, हो, सक...
1       [RT, @, Vishesh4, :, @, jawaharyadavbjp, जवाहर...
2       [RT, @, FunKeyBaat, :, #, भगवा, वस्त्र, पहन, क...
3       [Yey, nina, khothani, labafazi, benu, phambili...
4       [RT, @, Rajeshbhanjan2, :, जब, भी, कोई, सिकुलर...
                              ...                        
2958    [@, Varun_dvn, जिसके, नाम, के, साथ, इंडिअनस, ल...
2959    [मुझे, सबसे, ज्यादा, राहुल, और, सोनियाके, बारे...
2960    [RT, @, Gujju_Er, :, एक, आतंकी, जिसे, कांग्रेस...
2961    [RT, @, dayanand151, :, @, jafahar1, @, BBCHin...
2962    [RT, @, manakgupta, :, “, हमें, 5, साल, हो, गए...
Name: text, Length: 2963, dtype: object


In [19]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
print(length)
length=32

29.573449750382068


In [20]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [21]:
word_index=tokenizer.word_index

In [22]:
X_test=create_tokens(X_test)
print(X_test)

0      [@, Kinjal_Dubey_, इसके, लिए, तैयारी, हो, रखी,...
1      [RT, @, pratimamishra04, :, दिल्ली, के, मुख्यम...
2      [RT, @, nishantdabre2, :, आज, एक, तरफ, खुशी, ह...
3      [@, BJP4India, @, HardeepSPuri, जिस, कुनबे, के...
4                       [@, manakgupta, बादल, ..., .😂😂😂]
                             ...                        
658    [RT, @, AishwaryVerma9, :, दिल्ली, में, शून्य,...
659    [RT, @, KARUNASHANKEROJ, :, @, sagarikaghose, ...
660    [@, CM_P17, @, PShatrujeet, रमजान, शुरू, लेकिन...
661    [RT, @, rukasa007, :, तो, तुम्हारे, सिर, में, ...
662    [@, athavale_abhi, @, kavita_tewari, गांधी, पर...
Name: text, Length: 663, dtype: object


In [23]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  258,  4967,     9, ...,    21,     0,     0],
       [    4,     1,  4973, ...,     0,     0,     0],
       [ 1497,     2,    13, ...,   164,    51,   180],
       ...,
       [    4,     1, 14733, ...,     0,     0,     0],
       [    4,     1, 14736, ...,     0,     0,     0],
       [  301,   313,   287, ...,    18,     2,  2325]], dtype=int32)

In [24]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[   1,  299,   40, ...,    0,    0,    0],
       [   4,    1,    2, ...,    0,    0,    0],
       [   2,   56,   29, ...,  420,   37,    6],
       ...,
       [   1,    1, 4755, ...,    0,    0,    0],
       [   4,    1, 1219, ..., 4269,    0,    0],
       [   1,    1,  144, ...,    0,    0,    0]], dtype=int32)

In [25]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [26]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2963, 32), (663, 32), (2963, 4), (663, 4))

In [28]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Copy of crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [29]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [30]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  258  4967     9 ...    21     0     0]
 [    4     1  4973 ...     0     0     0]
 [ 1497     2    13 ...   164    51   180]
 ...
 [    4     1 14733 ...     0     0     0]
 [    4     1 14736 ...     0     0     0]
 [  301   313   287 ...    18     2  2325]]
[[   1  299   40 ...    0    0    0]
 [   4    1    2 ...    0    0    0]
 [   2   56   29 ...  420   37    6]
 ...
 [   1    1 4755 ...    0    0    0]
 [   4    1 1219 ... 4269    0    0]
 [   1    1  144 ...    0    0    0]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [31]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(14743, 300)

In [32]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 14743, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           4422900   
_________________________________________________________________
lstm (LSTM)                  (None, 192)               378624    
_________________________________________________________________
dense (Dense)                (None, 4)                 772       
Total params: 4,802,296
Trainable params: 379,396
Non-trainable params: 4,422,900
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=200,batch_size=256, verbose=2)

Epoch 1/200
12/12 - 2s - loss: 1.0134 - accuracy: 0.6615 - val_loss: 0.8771 - val_accuracy: 0.7436
Epoch 2/200
12/12 - 1s - loss: 0.8856 - accuracy: 0.7141 - val_loss: 0.8562 - val_accuracy: 0.7436
Epoch 3/200
12/12 - 1s - loss: 0.8767 - accuracy: 0.7141 - val_loss: 0.8765 - val_accuracy: 0.7436
Epoch 4/200
12/12 - 1s - loss: 0.8743 - accuracy: 0.7141 - val_loss: 0.8592 - val_accuracy: 0.7436
Epoch 5/200
12/12 - 1s - loss: 0.8778 - accuracy: 0.7141 - val_loss: 0.8688 - val_accuracy: 0.7436
Epoch 6/200
12/12 - 1s - loss: 0.8695 - accuracy: 0.7141 - val_loss: 0.8580 - val_accuracy: 0.7436
Epoch 7/200
12/12 - 1s - loss: 0.8670 - accuracy: 0.7141 - val_loss: 0.8642 - val_accuracy: 0.7436
Epoch 8/200
12/12 - 1s - loss: 0.8633 - accuracy: 0.7141 - val_loss: 0.8590 - val_accuracy: 0.7436
Epoch 9/200
12/12 - 1s - loss: 0.8596 - accuracy: 0.7141 - val_loss: 0.8545 - val_accuracy: 0.7436
Epoch 10/200
12/12 - 1s - loss: 0.8568 - accuracy: 0.7138 - val_loss: 0.8546 - val_accuracy: 0.7421
Epoch 11/

In [34]:
predictions_test = model.predict(X_test)

predictions_test

array([[9.07296169e-07, 9.98574376e-01, 1.34397515e-05, 1.41122902e-03],
       [1.25101160e-05, 3.57794692e-04, 4.46728154e-06, 9.99625206e-01],
       [7.89492130e-01, 1.56375691e-01, 3.37158745e-05, 5.40984944e-02],
       ...,
       [7.81415962e-03, 9.91676569e-01, 7.95993947e-06, 5.01352013e-04],
       [1.57924558e-04, 9.77584302e-01, 2.20342260e-02, 2.23538751e-04],
       [3.78388104e-05, 9.99937415e-01, 2.10166290e-05, 3.72095815e-06]],
      dtype=float32)

In [35]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
# output_DataFrame[0]=output_DataFrame[0].map({0:'NGEN',1:'GEN'})
# output_DataFrame.to_csv("German Task2 output.csv")

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.25      0.13      0.17        87
           1       0.77      0.86      0.81       493
           2       0.14      0.11      0.12        27
           3       0.06      0.05      0.06        56

   micro avg       0.67      0.67      0.67       663
   macro avg       0.31      0.29      0.29       663
weighted avg       0.62      0.67      0.64       663
 samples avg       0.67      0.67      0.67       663



In [37]:
  df3=df1[['tweet_id','ID']]
  df3

,tweet_id,ID
0,1127781647783301120,hasoc_2020_hi_1938
1,1127492853171535872,hasoc_2020_hi_1201
2,1123638375896903681,hasoc_2020_hi_1628
3,1127496460310683648,hasoc_2020_hi_3257
4,1127667277535399937,hasoc_2020_hi_4133
...,...,...
658,1123814066907242503,hasoc_2020_hi_466
659,1127616035689979904,hasoc_2020_hi_1758
660,1127502823044702208,hasoc_2020_hi_3521
661,1127780829914992640,hasoc_2020_hi_1293


In [38]:
predictions_test1=predictions_test.argmax(1)
pred_test1=pd.Series(predictions_test1)
pred_test1

0      1
1      3
2      0
3      1
4      1
      ..
658    1
659    1
660    1
661    1
662    1
Length: 663, dtype: int64

In [39]:
df3['label']=pred_test1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df3

,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,1
1,1127492853171535872,hasoc_2020_hi_1201,3
2,1123638375896903681,hasoc_2020_hi_1628,0
3,1127496460310683648,hasoc_2020_hi_3257,1
4,1127667277535399937,hasoc_2020_hi_4133,1
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,1
659,1127616035689979904,hasoc_2020_hi_1758,1
660,1127502823044702208,hasoc_2020_hi_3521,1
661,1127780829914992640,hasoc_2020_hi_1293,1


In [41]:
df3['label'] = df3['label'].map({0: "NONE",1:"OFFN",2:"PRFN",3:"HATE"})
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,OFFN
1,1127492853171535872,hasoc_2020_hi_1201,HATE
2,1123638375896903681,hasoc_2020_hi_1628,NONE
3,1127496460310683648,hasoc_2020_hi_3257,OFFN
4,1127667277535399937,hasoc_2020_hi_4133,OFFN
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,OFFN
659,1127616035689979904,hasoc_2020_hi_1758,OFFN
660,1127502823044702208,hasoc_2020_hi_3521,OFFN
661,1127780829914992640,hasoc_2020_hi_1293,OFFN


In [42]:
df3.to_csv('submission_HI_B.csv')

In [43]:
model.save("HindiDatasetSubTask2_withLSTM_Fasttext.h5")